In [18]:
import numba
import numpy as np

import ufl
from dolfinx import fem, mesh, io

from mpi4py import MPI
from petsc4py import PETSc

import sys
sys.path.append("../")
import fenicsx_support
import custom_assembling as ca


In [19]:
N = 2
domain = mesh.create_unit_square(MPI.COMM_WORLD, N, N)

deg, q_deg = 1, 2
V = fem.VectorFunctionSpace(domain, ("P", deg))

# quadrature elements and function spaces
QV = ufl.VectorElement(
    "Quadrature", domain.ufl_cell(), q_deg, quad_scheme="default", dim=3
)

QT = ufl.TensorElement(
    "Quadrature",
    domain.ufl_cell(),
    q_deg,
    quad_scheme="default",
    shape=(3, 3),
)
VQV = fem.FunctionSpace(domain, QV)
VQT = fem.FunctionSpace(domain, QT)

# define functions
u_, du = ufl.TestFunction(V), ufl.TrialFunction(V)
u = fem.Function(V)
q_sigma0 = fem.Function(VQV)
q_dsigma = fem.Function(VQT, name='stiffness')

num_dofs_global = V.dofmap.index_map.size_global * V.dofmap.index_map_bs
num_gauss_local = len(q_sigma0.x.array[:]) // 3
num_gauss_global = domain.comm.reduce(num_gauss_local, op=MPI.SUM, root=0)

# define form
dxm = ufl.dx(metadata={"quadrature_degree": q_deg, "quadrature_scheme": "default"})

def eps(u):
    e = ufl.sym(ufl.grad(u))
    return ufl.as_vector((e[0, 0], e[1, 1], 2 * e[0, 1]))

E, nu = 20000, 0.3

# Hookes law for plane stress
C11 = E / (1.0 - nu * nu)
C12 = C11 * nu
C33 = C11 * 0.5 * (1.0 - nu)
C = np.array([[C11, C12, 0.0], [C12, C11, 0.0], [0.0, 0.0, C33]], dtype=PETSc.ScalarType)

r"""
Set up 

  +---------+
/||         |->
/||         |->
/||         |-> u_bc
  o---------+
 / \
-----
"""


def left(x):
    return np.isclose(x[0], 0.0)


def right(x):
    return np.isclose(x[0], 1.0)


def origin(x):
    return np.logical_and(np.isclose(x[0], 0.0), np.isclose(x[1], 0.0))


u_bc = fem.Constant(domain, 0.0)  # expression for boundary displacement

dim = domain.topology.dim - 1
b_facets_l = mesh.locate_entities_boundary(domain, dim, left)
b_facets_r = mesh.locate_entities_boundary(domain, dim, right)
b_facets_o = mesh.locate_entities_boundary(domain, dim - 1, origin)

b_dofs_l = fem.locate_dofs_topological(V.sub(0), dim, b_facets_l)
b_dofs_r = fem.locate_dofs_topological(V.sub(0), dim, b_facets_r)
b_dofs_o = fem.locate_dofs_topological(V.sub(1), dim - 1, b_facets_o)

bcs = [
    fem.dirichletbc(PETSc.ScalarType(0), b_dofs_l, V.sub(0)),
    fem.dirichletbc(u_bc, b_dofs_r, V.sub(0)),
    fem.dirichletbc(PETSc.ScalarType(0), b_dofs_o, V.sub(1)),
]

In [20]:
bc = fem.dirichletbc(u_bc, b_dofs_r, V.sub(0))

In [21]:
bc.g.value

array(0.)

In [22]:
bc.dof_indices()

(array([ 2,  4, 10], dtype=int32), 3)

In [23]:
V.dofmap.cell_dofs(3)
dofmap = ca.get_topological_dofmap(V)
dofmap
# for cell in range(2):
#     pos = dofmap[cell]
    # u_local = u[pos]

array([[ 0,  1,  2,  3,  4,  5],
       [ 0,  1,  6,  7,  4,  5],
       [ 8,  9,  0,  1,  6,  7],
       [ 6,  7,  4,  5, 10, 11],
       [ 8,  9, 12, 13,  6,  7],
       [ 6,  7, 14, 15, 10, 11],
       [12, 13,  6,  7, 14, 15],
       [12, 13, 16, 17, 14, 15]], dtype=int32)

In [24]:
arr = dofmap[2]
ind = np.arange(6)
arr[ind]

array([8, 9, 0, 1, 6, 7], dtype=int32)

In [25]:
def sigma(u):
    return ufl.dot(eps(u), C_const)

def get_eval(self:ca.CustomFunction):
    tabulated_eps = self.tabulated_input_expression
    n_gauss_points = len(self.input_expression.X)
    stiffness = self.stiffness.value.reshape((n_gauss_points, -1))
    local_dim = self.local_dim
    
    @numba.njit
    def eval(values, coeffs_values, constants_values, coordinates, local_index, orientation):
        epsilon_local = np.zeros(local_dim, dtype=PETSc.ScalarType)
        sigma_local = values.reshape((n_gauss_points, -1))
        C_local = np.zeros(stiffness.shape, dtype=PETSc.ScalarType)

        tabulated_eps(ca.ffi.from_buffer(epsilon_local), 
                      ca.ffi.from_buffer(coeffs_values), 
                      ca.ffi.from_buffer(constants_values), 
                      ca.ffi.from_buffer(coordinates), ca.ffi.from_buffer(local_index), ca.ffi.from_buffer(orientation))
        
        epsilon_local = epsilon_local.reshape((n_gauss_points, -1))

        for i in range(n_gauss_points):
            sigma_local[i][:] = np.dot(stiffness[i].reshape((3,3)), epsilon_local[i]) 
        
        values = sigma_local.flatten()
        
        C_local = stiffness
        
        # print(C_local.shape)

        return [C_local.flatten()]
    return eval


In [26]:
from dolfinx import la
import typing

def get_dummy_x(V: fem.FunctionSpace) -> la.vector:
    dummy_domain = mesh.create_unit_square(MPI.COMM_WORLD, 1, 1)
    dummy_V = fem.FunctionSpace(dummy_domain, V._ufl_element)
    return la.vector(dummy_V.dofmap.index_map, dummy_V.dofmap.index_map_bs)

class DummyFunction(fem.Function): #or ConstantFunction?
    def __init__(self, V: fem.FunctionSpace, name: typing.Optional[str] = None ):
        super().__init__(V=V, x=get_dummy_x(V), name=name)
        self.value = self.x.array.reshape((2, -1))[0]
    
    def fill(self, value: np.ndarray):
        self.value[:] = value

In [27]:
q_dsigma = DummyFunction(VQT, name='stiffness')
n_gauss_nodes = 3
new_values = np.tile(C.flatten(), n_gauss_nodes)
q_dsigma.fill(new_values)
q_dsigma.value.reshape((n_gauss_nodes, *C.shape))

array([[[21978.02197802,  6593.40659341,     0.        ],
        [ 6593.40659341, 21978.02197802,     0.        ],
        [    0.        ,     0.        ,  7692.30769231]],

       [[21978.02197802,  6593.40659341,     0.        ],
        [ 6593.40659341, 21978.02197802,     0.        ],
        [    0.        ,     0.        ,  7692.30769231]],

       [[21978.02197802,  6593.40659341,     0.        ],
        [ 6593.40659341, 21978.02197802,     0.        ],
        [    0.        ,     0.        ,  7692.30769231]]])

In [28]:
q_sigma = ca.CustomFunction(VQV, eps(u), [q_dsigma], get_eval)
# q_dsigma = fem.Function(VQT)
# print(q_dsigma.x.array[:].shape)
# # q_dsigma = fem.Function(VQT, get_dummy_x(VQT), name='stiffness')
# print(q_dsigma.x.array[:].shape)
# q_sigma.add_coefficient(q_dsigma)
q_sigma.coefficients

[Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 2), TensorElement(FiniteElement('Quadrature', triangle, 2, quad_scheme='default'), shape=(3, 3), symmetry={})), 140071013495424)]

In [29]:
C_const = fem.Constant(domain, C)
# q_dsigma = fem.Function(VQT, name='stiffness')
# q_dsigma.x.array[:] = np.tile(C.flatten(), 24)
# q_sigma = fem.Function(VQV)

R = ufl.inner(eps(u_), q_sigma) * dxm
dR = ufl.inner(eps(du), ufl.dot(q_sigma.stiffness, eps(u_))) * dxm

R0 = ufl.inner(eps(u_), q_sigma0) * dxm
dR0 = ufl.inner(eps(du), ufl.dot(C_const, eps(u_))) * dxm

In [30]:
# bc_indexes = np.sort(np.concatenate([bc.dof_indices()[0] for bc in bcs]) )

In [31]:
# bb = fem.petsc.create_vector(fem.form(R0))
# with bb.localForm() as b_local:
#     b_local.set(0.0)

# gg = fem.petsc.create_vector(fem.form(R0))
# with gg.localForm() as b_local:
#     b_local.set(0.0)

# g0 = fem.petsc.create_vector(fem.form(R0))
# with g0.localForm() as b_local:
#     b_local.set(19.0)

# xx0 = fem.petsc.create_vector(fem.form(R0))
# with g0.localForm() as b_local:
#     b_local.set(0.0)

# A_brut = fem.petsc.create_matrix(fem.form(dR0))
# A_brut.zeroEntries()
A0 = fem.petsc.create_matrix(fem.form(dR0))
A0.zeroEntries()
b0 = fem.petsc.create_vector(fem.form(R0))
with b0.localForm() as b_local:
    b_local.set(0.0)
u0 = fem.Function(V)

# q_sigma.vector.set(0)
q_sigma0.vector.set(0)

solver0 = PETSc.KSP().create(domain.comm)
solver0.setType("preonly")
solver0.getPC().setType("lu")

# f = io.XDMFFile(domain.comm, "displacements.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5)
# f.write_mesh(domain)

u = fem.Function(V)
u.name = "Displacement"
my_solver = ca.CustomSolver(dR, R, u, bcs)
scale = -1.0

u_bc_max = 42.0
ts = np.linspace(0.0, 1.0, 5)
for t in ts:
    # update value of Dirichlet BC
    u_bc.value = t * u_bc_max

    print(f"Solving {t = :6.3f} with {u_bc.value = :6.3f}...")

    eps_calculated = fenicsx_support.interpolate_quadrature(eps(u0), q_deg, domain)
    strain_matrix = eps_calculated.reshape((-1, 3))
    n_gauss = len(strain_matrix) #global in the domain

    q_sigma0.x.array[:] = (strain_matrix @ C).flatten()
    # q_sigma.x.array[:] = (strain_matrix @ C).flatten()
    
    q_sigma0.x.scatter_forward()
    # q_sigma.x.scatter_forward()

    # q_dsigma.x.array[:] = np.tile(C.flatten(), n_gauss)
    # print('q_sigma0\n', q_sigma0.x.array[:])
    # update matrix (pointless in linear elasticity...)
    A0.zeroEntries()
    # A_brut.zeroEntries()
    # update residual vector

    # with bb.localForm() as b_local:
    #     b_local.set(0.0)

    # with g0.localForm() as b_local:
    #     b_local.set(0.0)

    # with gg.localForm() as b_local:
    #     b_local.set(0.0)

    with b0.localForm() as b_local:
        b_local.set(0.0)

    fem.petsc.assemble_matrix(A0, fem.form(dR0), bcs=bcs)
    A0.assemble()

    fem.petsc.assemble_vector(b0, fem.form(R0))
    # print(A0[:,:])
    # print('b0 assembled\n', b0[:],'\n')


    fem.apply_lifting(b0, [fem.form(dR0)], bcs=[bcs], x0=[u0.vector], scale=-1.0)
    # print('b0 lifted\n', b0[:],'\n')

    b0.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

    fem.set_bc(b0, bcs, u0.vector, -1.0)
    # print('g0 - u0\n',g0[:] - u0.vector[:],'\n')
    # print('b0 lifted\n', b0[:],'\n')

    # fem.petsc.assemble_vector(g0, fem.form(R0))
    # # print('g0 assembled\n', g0[:],'\n')
    # fem.petsc.assemble_matrix(A_brut, fem.form(dR0))
    # A_brut.assemble()
    
    # fem.apply_lifting(g0, [fem.form(dR0)], bcs=[bcs], x0=[u0.vector], scale=scale)
    # # print('g0 lifted\n', g0[:], '\n')

    # # print('u0\n', u0.vector[:])
    # fem.set_bc(g0, bcs, x0=u0.vector, scale=scale)
    # # print('g0\n',g0[:],'\n')

    # fem.petsc.assemble_vector(bb, fem.form(R0))


    # x0 = np.zeros_like(u0.vector[:])
    # for i, this_u0 in enumerate(u0.vector[:]):
    #     if i in bc_indexes:
    #         x0[i] = this_u0
    # # print(x0)
    # fem.set_bc(gg, bcs)
    # fem.set_bc(xx0, bcs, gg[:] + u0.vector[:], scale=-1.)
    # print(gg[:])
    # print(xx0[:])
    # print('Agg\n', bb[:] - A_brut[:,:] @ gg[:], '\n') # gg - changes each time step as bc condition 
    # print('Agg - g0\n', A_brut[:,:] @ (gg[:]) - g0[:], '\n')
    # print('Agg - g0\n', bb[:] - scale * A_brut[:,:] @ (gg[:] - x0[:]), '\n')
    # print(my_solver.b[:])


    solver0.setOperators(A0)
    du0 = fem.Function(V)  # Should be outside of loop, instructive here.
    solver0.solve(b0, du0.vector)
    
    # Solve for the displacement increment du, apply it and udpate ghost values
    du = fem.Function(V)  # Should be outside of loop, instructive here.
    my_solver.solve(du, x0=u0.x.array, scale=scale)
    # print(my_solver.b[:] - g0[:])


    u0.x.array[:] -= du0.x.array[:]
    u0.x.scatter_forward()
    
    u.x.array[:] -= du.x.array[:]
    u.x.scatter_forward()

    # print(f'rank = {MPI.COMM_WORLD.rank} u - u0\n {u.x.array[:]} \n {u0.x.array[:]} \n')
    print(f'rank = {MPI.COMM_WORLD.rank} u - u0\n {np.linalg.norm(u.x.array[:] - u0.x.array[:])} \n')
    # assert np.linalg.norm(u.x.array[:] - u0.x.array[:]) < 1.0e-10
    # print(f'rank = {MPI.COMM_WORLD.rank} q_sigma - q_sigma0\n {np.linalg.norm(q_sigma.x.array[:] - q_sigma0.x.array[:])} \n')

    # print(f'{q_sigma.x.array}')
    # print(f'{q_sigma0.x.array}')
    # post processing
    # f.write_function(u, t)

# f.close()

Solving t =  0.000 with u_bc.value =  0.000...
rank = 0 u - u0
 0.0 

Solving t =  0.250 with u_bc.value = 10.500...
rank = 0 u - u0
 0.0 

Solving t =  0.500 with u_bc.value = 21.000...
rank = 0 u - u0
 1.8815947990869657e-14 

Solving t =  0.750 with u_bc.value = 31.500...
rank = 0 u - u0
 3.5495863371570936e-14 

Solving t =  1.000 with u_bc.value = 42.000...
rank = 0 u - u0
 4.74774028131087e-14 

